In [1]:
%autosave 20

import pandas as pd
import numpy as np

# print all the outputs in a cell
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

Autosaving every 20 seconds


## Download Data

### Function1 - Transfer csv to formal table

In [2]:
## transfer csv to formal table

def google_to_table(cur_csv):
    
    # 1. create list for region need 
    region_need = ['San Diego CA', 
               'Los Angeles CA', 
               'San Francisco-Oakland-San Jose CA', 
               'Austin TX', 
               'Dallas-Ft. Worth TX', 'San Antonio TX', 'Sacramento-Stockton-Modesto CA']
    
    cur_csv = cur_csv.reset_index()
    cur_csv.rename(columns={'index':'Region','Category: All categories':'Search'}, inplace=True)
    
    
    # 2. create Year, Month by datetime

    # 2.1 transfer str to datetime 
    from datetime import datetime
    date_string = cur_csv.iloc[0,1][-9:-1]
    #del space
    date_string = date_string.strip()
    
    
    cur_csv['datetime'] = datetime.strptime(date_string, '%m/%d/%y')

    # 2.2 create Year, Month by datetime
    cur_csv['Year'] = cur_csv['datetime'].dt.year
    cur_csv['Month'] = cur_csv['datetime'].dt.month
    
    
    
    # 3. choose rows by region_need - only 6 
    idx = []

    for i in range(len(cur_csv)):
        for j in range(len(region_need)):
            if cur_csv.iloc[i, 0] == region_need[j]:
                idx.append(i)

    cur_csv = cur_csv.loc[(idx)]

    # 4. fillna search with nan
    cur_csv.fillna(0, inplace=True)

    # 5. special case, when search = <1
    for i in range(len(cur_csv)):
        if cur_csv.iloc[(i, 1)] == '<1':
            cur_csv.iloc[(i, 1)] = 0
            
    # 6. adjust dtype for columns
    cur_csv['Search'] = cur_csv['Search'].astype('int32')
    #cur_csv.drop(columns='datetime', inplace=True)
    
    return cur_csv

### Function2 - Put all csv files into one list

In [3]:
# put all csv file into one list

import os
import csv

nameforfolders = ['calfresh_2022', 'foodbank_2022', 'foodpantry_2022', 'foodstamp_2022', 'supplemental nutrition assistance program_2022']
lookup = {}


for googleword in nameforfolders:
    lookup[googleword] = None
    
    #each google word has one folder
    folder_path = '/Users/chensumei/Desktop/Ram Paper/2022 Data/GoogleTrend/' + googleword

    # each folder have 12 files
    # List all files in the folder
    files = os.listdir(folder_path)
    print('The size of ' + str(files) + 'is: ' + str(len(files)))
    print(' ')

    # list to save the names of 12 files 
    tem = []

    # Iterate over the files
    for file in files:
        file_path = os.path.join(folder_path, file)  # Create the full file path
        test = pd.read_csv(file_path)
        tem.append(test)
        
    lookup[googleword] = tem

The size of ['calfresh_2022_09.csv', 'calfresh_2022_08.csv', 'calfresh_2022_05.csv', 'calfresh_2022_11.csv', 'calfresh_2022_10.csv', 'calfresh_2022_04.csv', 'calfresh_2022_12.csv', 'calfresh_2022_06.csv', 'calfresh_2022_07.csv', 'calfresh_2022_03.csv', 'calfresh_2022_02.csv', 'calfresh_2022_01.csv']is: 12
 
The size of ['foodbank_2022_07.csv', 'foodbank_2022_12.csv', 'foodbank_2022_06.csv', 'foodbank_2022_10.csv', 'foodbank_2022_04.csv', 'foodbank_2022_05.csv', 'foodbank_2022_11.csv', 'foodbank_2022_01.csv', 'foodbank_2022_02.csv', 'foodbank_2022_03.csv', 'foodbank_2022_08.csv', 'foodbank_2022_09.csv']is: 12
 
The size of ['foodpantry_2022_08.csv', 'foodpantry_2022_09.csv', 'foodpantry_2022_010.csv', 'foodpantry_2022_02.csv', 'foodpantry_2022_03.csv', 'foodpantry_2022_01.csv', 'foodpantry_2022_04.csv', 'foodpantry_2022_05.csv', 'foodpantry_2022_11.csv', 'foodpantry_2022_07.csv', 'foodpantry_2022_12.csv', 'foodpantry_2022_06.csv']is: 12
 
The size of ['foodstamp_2022_03.csv', 'foodstamp

### Create a empty dataframe to save the cleaning table

In [4]:
#create a empty dataframe to save the cleaning table
df = pd.DataFrame()

for word in nameforfolders:
    # name of the word - no need '_2022'
    x_name = str(word)[0:-5]
    
    for j in range(12):
        # table of the word
        x = lookup[word][j]

        # after cleaning
        y = google_to_table(x)

        # add column 'Searchword' to the table
        y['Searchterm'] = x_name

        df = pd.concat([df, y], join='outer')

In [5]:
df = df.reset_index().drop(columns='index')

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 420 entries, 0 to 419
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   Region      420 non-null    object        
 1   Search      420 non-null    int32         
 2   datetime    420 non-null    datetime64[ns]
 3   Year        420 non-null    int64         
 4   Month       420 non-null    int64         
 5   Searchterm  420 non-null    object        
dtypes: datetime64[ns](1), int32(1), int64(2), object(2)
memory usage: 18.2+ KB


In [7]:
df.Region.unique()

array(['Sacramento-Stockton-Modesto CA', 'Los Angeles CA', 'San Diego CA',
       'San Francisco-Oakland-San Jose CA', 'Dallas-Ft. Worth TX',
       'Austin TX', 'San Antonio TX'], dtype=object)

In [8]:
for i in range(len(df)):
    if df.iloc[i, 0] == 'Dallas-Ft. Worth TX':
        df.iloc[i, 0] = 'DFW'
        
    elif df.iloc[i, 0] == 'Austin TX':
        df.iloc[i, 0] = 'Austin'
        
    elif df.iloc[i, 0] == 'San Antonio TX':
        df.iloc[i, 0] = 'SanAntonio'
        
    else:
        pass

In [9]:
df

,Region,Search,datetime,Year,Month,Searchterm
0,Sacramento-Stockton-Modesto CA,61,2022-09-30,2022,9,calfresh
1,Los Angeles CA,57,2022-09-30,2022,9,calfresh
2,San Diego CA,56,2022-09-30,2022,9,calfresh
3,San Francisco-Oakland-San Jose CA,32,2022-09-30,2022,9,calfresh
4,DFW,1,2022-09-30,2022,9,calfresh
5,Austin,0,2022-09-30,2022,9,calfresh
6,SanAntonio,0,2022-09-30,2022,9,calfresh
7,San Diego CA,49,2022-08-31,2022,8,calfresh
8,Sacramento-Stockton-Modesto CA,48,2022-08-31,2022,8,calfresh
9,Los Angeles CA,46,2022-08-31,2022,8,calfresh


In [10]:
# out put the table

df.to_excel('GoogleTrend_2022.xlsx')